In [1]:
import logging
import platform, psutil
import math

from fimdpenv import setup
from fimdpenv.UUVEnv import SingleAgentEnv
from fipomdp import ConsPOMDP
from fipomdp.pomcp import OnlineStrategy

setup()
logging_level = logging.WARNING

## UUV POMDP Single Agent

Run following 2 cells for logging

In [2]:
log_file_name = "UUVExperiments"  # Change for your needs
logging_level = logging.INFO # set to INFO for logging to be active

In [3]:
logging.basicConfig(filename=f"{log_file_name}.log",
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S')
logging.info('START')

Device logs

In [4]:
uname = platform.uname()
print(f"Node name: {uname.node}")
print(f"System: {uname.system}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")
print(f"RAM: {str(round(psutil.virtual_memory().total / (1024.0 **3)))} GB")

Node name: simonbrlej-ThinkPad-T14-Gen-1
System: Linux
Release: 5.11.0-44-generic
Version: #48~20.04.2-Ubuntu SMP Tue Dec 14 15:36:44 UTC 2021
Machine: x86_64
Processor: x86_64
RAM: 15 GB


##### Create environment with observations

In [5]:
from fipomdp.environment_utils import set_cross_observations_to_UUV_grid

In [6]:
logging.info('Creating UUV environment with observations.')

env = SingleAgentEnv(grid_size=[20, 20], capacity=20, reloads=[64, 69, 74, 164, 169, 174, 264, 269, 274, 364, 369, 374], targets=[103, 209, 210, 270], init_state=399, enhanced_actionspace=0)

mdp, targets = env.get_consmdp()
mdp.__class__ = ConsPOMDP
set_cross_observations_to_UUV_grid(mdp, (env.grid_size[0], env.grid_size[1]))

logging.info('Environment created')

##### Calculate belief support cmdp and guessing cmdp

In [9]:
cpomdp = mdp
capacity = env.capacities[0]
init_energy = capacity
init_obs = 399
init_bel_supp = tuple([399])
exploration = 0.9
random_seed = 1

cpomdp.compute_guessing_cmdp_initial_state([399])
strategy = OnlineStrategy(cpomdp, capacity, init_energy, init_obs, init_bel_supp, targets, exploration, random_seed, False)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
strategy.tree.action_shield

In [ ]:
import math

env = SingleAgentEnv(grid_size=[2, 2], capacity=20, reloads=[0], targets=[2], init_state=0, enhanced_actionspace=0)

mdp, targets = env.get_consmdp()
mdp.__class__ = ConsPOMDP
set_cross_observations_to_UUV_grid(mdp, (env.grid_size[0], env.grid_size[1]))

cpomdp = mdp
capacity = env.capacities[0]
init_energy = capacity
init_obs = 0
init_bel_supp = tuple([0])
exploration = 0.9
random_seed = 1

strategy = OnlineStrategy(cpomdp, capacity, init_energy, init_obs, init_bel_supp, targets, exploration, random_seed, logged, True)
print(strategy.tree.action_shield)
print(env.capacities[0])
print(init_energy)

In [ ]:
from fimdp.objectives import BUCHI, SAFE, POS_REACH
strategy.solver.guess_min_levels[BUCHI]



In [ ]:
strategy.next_action(10)

In [ ]:
strategy.update_obs(1)

In [ ]:
strategy.tree.cpomdp.get_state_obs_probs(1)

In [ ]:
from collections import deque

queue = deque()
queue.append(strategy.tree.root)
while len(queue) > 0:
    node = queue.popleft()
    for child in node.children:
        queue.append(child)
        print(child.visits, child.val)
        print()

In [ ]:
from fipomdp.energy_solvers import ConsPOMDPBasicES

solver = ConsPOMDPBasicES(cpomdp, [0], capacity, targets, False)
solver.compute_posreach()
solver.compute_buchi()

In [ ]:


solver.compute_safe()
solver.guess_min_levels[SAFE]